<a href="https://colab.research.google.com/github/Dhanush-adk/intro_to_dl/blob/main/Assignment_5/Assignment_5_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.33 s (started: 2024-04-27 01:55:56 +00:00)


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import random
from torch.nn.utils.rnn import pad_sequence


time: 3.53 s (started: 2024-04-27 01:56:15 +00:00)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

time: 5.78 ms (started: 2024-04-27 01:56:19 +00:00)


In [5]:
english_to_french = [
    ("I am cold", "J'ai froid"),

    ("You are tired", "Tu es fatigué"),

    ("He is hungry", "Il a faim"),

    ("She is happy", "Elle est heureuse"),

    ("We are friends", "Nous sommes amis"),

    ("They are students", "Ils sont étudiants"),

    ("The cat is sleeping", "Le chat dort"),

    ("The sun is shining", "Le soleil brille"),

    ("We love music", "Nous aimons la musique"),

    ("She speaks French fluently", "Elle parle français couramment"),

    ("He enjoys reading books", "Il aime lire des livres"),

    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),

    ("The movie starts at 7 PM", "Le film commence à 19 heures"),

    ("She wears a red dress", "Elle porte une robe rouge"),

    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),

    ("He drives a blue car", "Il conduit une voiture bleue"),

    ("They visit museums often", "Ils visitent souvent des musées"),

    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),

    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),

    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),

    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),

    ("They travel around the world", "Ils voyagent autour du monde"),

    ("The book is on the table", "Le livre est sur la table"),

    ("She dances gracefully", "Elle danse avec grâce"),

    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),

    ("He works hard every day", "Il travaille dur tous les jours"),

    ("They speak different languages", "Ils parlent différentes langues"),

    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),

    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),

    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),

    ("The dog barks loudly", "Le chien aboie bruyamment"),

    ("He sings beautifully", "Il chante magnifiquement"),

    ("They swim in the pool", "Ils nagent dans la piscine"),

    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),

    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),

    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),

    ("He paints landscapes", "Il peint des paysages"),

    ("They laugh at the joke", "Ils rient de la blague"),

    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),

    ("She runs in the park", "Elle court dans le parc"),

    ("We travel by train", "Nous voyageons en train"),

    ("He writes a letter", "Il écrit une lettre"),

    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),

    ("The baby cries", "Le bébé pleure"),

    ("She studies hard for exams", "Elle étudie dur pour les examens"),

    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),

    ("He fixes the car", "Il répare la voiture"),

    ("They drink coffee in the morning", "Ils boivent du café le matin"),

    ("The sun sets in the evening", "Le soleil se couche le soir"),

    ("She dances at the party", "Elle danse à la fête"),

    ("We play music at the concert", "Nous jouons de la musique au concert"),

    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),

    ("They study French grammar", "Ils étudient la grammaire française"),

    ("The rain falls gently", "La pluie tombe doucement"),

    ("She sings a song", "Elle chante une chanson"),

    ("We watch a movie together", "Nous regardons un film ensemble"),

    ("He sleeps deeply", "Il dort profondément"),

    ("They travel to Paris", "Ils voyagent à Paris"),

    ("The children play in the park", "Les enfants jouent dans le parc"),

    ("She walks along the beach", "Elle se promène le long de la plage"),

    ("We talk on the phone", "Nous parlons au téléphone"),

    ("He waits for the bus", "Il attend le bus"),

    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),

    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),

    ("She dreams of flying", "Elle rêve de voler"),

    ("We work in the office", "Nous travaillons au bureau"),

    ("He studies history", "Il étudie l'histoire"),

    ("They listen to the radio", "Ils écoutent la radio"),

    ("The wind blows gently", "Le vent souffle doucement"),

    ("She swims in the ocean", "Elle nage dans l'océan"),

    ("We dance at the wedding", "Nous dansons au mariage"),

    ("He climbs the mountain", "Il gravit la montagne"),

    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),

    ("The cat meows loudly", "Le chat miaule bruyamment"),

    ("She paints a picture", "Elle peint un tableau"),

    ("We build a sandcastle", "Nous construisons un château de sable"),

    ("He sings in the choir", "Il chante dans le chœur")
]

time: 5.8 ms (started: 2024-04-27 01:56:19 +00:00)


In [6]:
# Assume english_to_french is defined elsewhere
french_to_english = [(french, english) for english, french in english_to_french]
word_to_index = {"SOS": 0, "EOS": 1}
for eng, fre in english_to_french:
    for word in eng.split() + fre.split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

time: 982 µs (started: 2024-04-27 01:56:34 +00:00)


In [7]:
# Dataset class for translation data
class TranslationDataset(Dataset):
    def __init__(self, dataset, word_to_index):
        self.dataset = dataset
        self.word_to_index = word_to_index

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        french_sentence, english_sentence = self.dataset[idx]
        input_indices = [self.word_to_index[word] for word in french_sentence.split()] + [word_to_index["EOS"]]
        target_indices = [self.word_to_index[word] for word in english_sentence.split()] + [word_to_index["EOS"]]
        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_indices, dtype=torch.long)

def collate_batch(batch):
    input_tensors, target_tensors = zip(*batch)
    input_tensors_padded = pad_sequence(input_tensors, batch_first=True, padding_value=word_to_index["EOS"])
    target_tensors_padded = pad_sequence(target_tensors, batch_first=True, padding_value=word_to_index["EOS"])
    return input_tensors_padded, target_tensors_padded

# Transformer Model
class Transformer(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers = 4, num_heads = 8, dropout = 0.2):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.transformer = nn.Transformer(
            d_model=hidden_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_size * 4,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(hidden_size, vocab_size)

    def forward(self, src, tgt):
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)
        output = self.transformer(src, tgt, tgt_mask=tgt_mask)
        return self.fc_out(output)


time: 2.35 ms (started: 2024-04-27 01:56:45 +00:00)


In [8]:
# Initialization and Running Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = len(word_to_index)
embed_size = 128
num_layers = 4
num_heads = 8
dropout = 0.2
epochs = 60

time: 990 µs (started: 2024-04-27 01:56:48 +00:00)


In [9]:
model = Transformer(input_size, embed_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_function = nn.CrossEntropyLoss(ignore_index=word_to_index["EOS"])

dataset = TranslationDataset(french_to_english, word_to_index)
translation_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn= collate_batch)

for epoch_num in range(epochs + 1):
    model.train()

    total_train_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for english_tensor, french_tensor in translation_loader:
        english_tensor = english_tensor.to(device)
        french_tensor = french_tensor.to(device)

        optimizer.zero_grad()
        output = model(english_tensor, french_tensor[:, :-1])
        output_flat = output.view(-1, output.size(-1))
        french_target_flat = french_tensor[:, 1:].contiguous().view(-1)

        valid_french_tokens = french_target_flat != 1
        loss = loss_function(output_flat[valid_french_tokens], french_target_flat[valid_french_tokens])
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(output_flat, 1)
        valid_predictions = (predicted == french_target_flat) & valid_french_tokens
        correct_predictions += valid_predictions.sum().item()
        total_predictions += valid_french_tokens.sum().item()

    mean_train_loss = total_train_loss / len(translation_loader)
    training_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

    model.eval()
    total_eval_loss = 0
    eval_correct = 0
    eval_total = 0
    with torch.no_grad():
        for english_tensor, french_tensor in translation_loader:
            english_tensor = english_tensor.to(device)
            french_tensor = french_tensor.to(device)

            output = model(english_tensor, french_tensor[:, :-1])
            output_flat = output.view(-1, output.size(-1))
            french_target_flat = french_tensor[:, 1:].contiguous().view(-1)

            valid_french_tokens = french_target_flat != 1
            loss = loss_function(output_flat[valid_french_tokens], french_target_flat[valid_french_tokens])

            total_eval_loss += loss.item()

            _, predicted = torch.max(output_flat, 1)
            valid_predictions = (predicted == french_target_flat) & valid_french_tokens
            eval_correct += valid_predictions.sum().item()
            eval_total += valid_french_tokens.sum().item()

    mean_eval_loss = total_eval_loss / len(translation_loader)
    evaluation_accuracy = eval_correct / eval_total if eval_total > 0 else 0
    if epoch_num % 10 == 0:
        print(f'Epoch {epoch_num}: Loss = {mean_train_loss:.4f}, Train Accuracy = {training_accuracy:.4f}, '
              f'Val Loss = {mean_eval_loss:.4f}, Val Accuracy = {evaluation_accuracy:.4f}')


Epoch 0: Loss = 5.8832, Train Accuracy = 0.0440, Val Loss = 5.4684, Val Accuracy = 0.0916
Epoch 10: Loss = 4.5603, Train Accuracy = 0.1136, Val Loss = 4.1739, Val Accuracy = 0.1502
Epoch 20: Loss = 3.4737, Train Accuracy = 0.2784, Val Loss = 2.8000, Val Accuracy = 0.4469
Epoch 30: Loss = 2.5417, Train Accuracy = 0.5458, Val Loss = 1.8787, Val Accuracy = 0.6923
Epoch 40: Loss = 1.7462, Train Accuracy = 0.7436, Val Loss = 1.1752, Val Accuracy = 0.8755
Epoch 50: Loss = 1.2594, Train Accuracy = 0.8901, Val Loss = 0.6973, Val Accuracy = 0.9890
Epoch 60: Loss = 0.8178, Train Accuracy = 0.9780, Val Loss = 0.3975, Val Accuracy = 1.0000
time: 28.5 s (started: 2024-04-27 01:57:38 +00:00)


In [11]:
sentence = "Elle est heureuse"

model.eval()  # Set the model to evaluation mode

# Tokenize the sentence and convert each word to its corresponding index
words = sentence.split()
input_tensor = torch.tensor([word_to_index[word] for word in words if word in word_to_index]).unsqueeze(0).to(device)

# Initialize the output tensor with the index of "SOS" (start of sentence)
output_indices = [word_to_index['SOS']]
for _ in range(len(words)):
    # Convert the current output indices to a tensor input for the model
    output_tensor = torch.tensor(output_indices).unsqueeze(0).to(device)

    # Get predictions from the model
    with torch.no_grad():
        output = model(input_tensor, output_tensor)

    # Find the predicted word (index with the highest probability in the last position)
    predicted_index = output[0, -1, :].argmax(0).item()

    # If "EOS" (end of sentence), stop the prediction
    if predicted_index == word_to_index['EOS']:
        break

    # Append the predicted index to the output sequence
    output_indices.append(predicted_index)

# Convert indices back to words
index_to_word = {index: word for word, index in word_to_index.items()}
translated_sentence = ' '.join([index_to_word[idx] for idx in output_indices if idx != word_to_index['SOS']])
translated_sentence

'the is happy'

time: 72.6 ms (started: 2024-04-27 01:59:39 +00:00)
